<a href="https://colab.research.google.com/github/geocarvalho/uni-proj/blob/master/IF699/cleber/5-otimizacao_de_sistemas_parametricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Otimização de sistemas paramétricos

## Data preprocessing

`original data > zero-centered data > normalized data`

* **Antes de normalizar**: Perda de classificação muito sensível a mundaças na matrix de peso, difícil de otimizar;
  
* **Depois da normalização**: Menos sensível a pequenas mudanças nos pesos, fácil de otimizar.
  
## Onde paramos anteriormente:

* função de pontuação: $s = f(x; W) = W_x$

* função de perda (SVM): $L_i = \sum_{j \neq y_i} max(0, s_j - s_{y_i} + 1)$

* função de perda + regularização: $L = \frac{1}{N} \sum^{N}_{i=1} L_i + \sum_k W^2_k$

* queremos $\nabla_w L$

## *Batch gradient descent* 

* *Vanilla gradient descent* computa o gradiente da função de custo para os parâmetros para todo o dado de treinamento

$\theta = \theta - \eta \times \nabla_{\theta}J(\theta)$



In [0]:
def exemplo(args):
  for i in range(nb_epochs):
    params_grad = evaluate_gradient(loss_function, data, params)
    params = params - learning_rate * params_grad

## *Stochastic gradient descent* (SGD)

* Gradiente descedente estocástico atualiza os parametros para cada exemplo de treinamento $x(i)$ e rótulo $y(i)$:

$\theta = \theta - \eta \times \nabla_{\theta}J(\theta, x^{(i)}; y^{(i)})$


In [0]:
def exemplo(args):
  for i in range(nb_epochs):
    np.random.shuffle(data)
    for example in data:
      params_grad = evaluate_gradient(loss_function, example, params)
      params = params - learning_rate * params_grad

## *Mini-batch gradient descent*

* Gradiente descendente por mini-lote escolhe o melhor dos dois mundos e atualiza para mini-lote de $n$ exemplos de treino:

$\theta = \theta - \eta \times \nabla_{\theta}J(\theta, x^{(i:i+n)}; y^{(i:i+n)})$

In [0]:
def exemplo(args):
  for i in range(nb_epochs):
    np.random.shuffle(data)
    for batch in get_batches(data, batch_size=50):
      params_grad = evaluate_gradient(loss_function, batch, params)
      params = params - learning_rate * params_grad

## Problemas com SGD

* O que ocorre com mudanças de perda rápidas numa direção e lentas em outra? O que o gradiente descendente faz?

> A função de perda tem alta **condição de número**: proporção de valores singulares grandes para pequenos da matrix Hession é largo.

* O que ocorre se a função de perda tem um mínimo local ou um ponto de sela?

> Gradiente zero, gradiente descendente fica travado. Pontos de sela são comuns em altas dimensões.

* O gradiente vem do mini-batch por isso é pode ter alta variação (*noisy*)

![noisy](noisy.png)

$L(W) = \frac{1}{N} \sum^N_{i=1} L_i(x_i, y_i, W)$


$\nabla_W L(W) = \frac{1}{N} \sum^N_{i=1} \nabla_WL_i(x_i, y_i, W)$

## SGD + momentum

* SGD $x_{t+1} = x_t - \alpha\nabla f(x_t)$

In [0]:
def sgd(args):
  while True:
    dx = compute_gradient(x)
    x += learning_rate * dx

* SGD + Momentum

$v_{t+1} = pv_t + \nabla f(x_t)$

$x_{t+1} = x_t - \alpha v_{t+1}$

In [0]:
def sgd_momentum(args):
  vx = 0
  while True:
    dx = compute_gradient(x)
    vx = rho * vx + dx
    x += learning_rate * vx

* Construa "velocidade" rodando a média dos gradientes;
* `Rho` dá "fricção", tipicamente rho = .9 ou .99
* Se distancia do mínimo local ou do ponto de sela

![gradient_noise](gradient_noise.png)

![momentum_update](momentum_update.png)

## *Nesterov momentum*

![nesterov](nesterov.png)

## NAG vs. *Standard momentum*

* Primeiro faça um grande puki na direção do grandiente acumulado anteriormente. Então calcule o gradiente onde você termina e faça a correção.

![jump](jump.png)

## De volta ao *Nesterov momentum*

$v_{t+1} = \rho v_t - \alpha \nabla f(x_t + \rho v_t)$

$x_{t+1} = x_t + v_{t+1}$

> onde $x_t + pv_t$ normalmente queremos atualizar em termos de $x_t, \nabla f(x_t)$.

* Mudamos as variáveis $\tilde{x}_t + x-t + pv_t$ e rearranjamos.

$v{t+1} = \rho v_t - \alpha \nabla f(\tilde{x}_t)$

então,

$\tilde{x}_{t+1} = \tilde{x}_t +v_{t+1} + \rho(v_{t+1} - v_t)$


In [0]:
def nesterov(args):
  dx = compute_gradient(x)
  old_v = v
  v = rho * v - learning_rate * dx
  x += -rho * old_v + (1 + rho) * v

* Por que nada novo (entre *momentum*/NAG)?

> Como indicar a taxa de aprendizado e declínio da taxa de aprendizado?

> O ideal são as taxa de aprendizado adaptativo.

## Propagação resiliente (Rprop)

* Riedmiller e Braun, 1993;

* Derecionado ao problema da taxa de aprendizado adaptativo

* Aumenta a taxa de aprendizado por pesos multiplicativamente se os sinais dos dois gradientes anteriores concordarem. Senão, diminui a taxa de aprendizado multiplicativamente.

## Rprop update

* if $f'_t f'_{t-1} > 0: v_t = \eta^+ v{t-1}$

* else if $f'_t f'_{t-1} < 0: v_t = \eta^- v_{t-1}$

* else: $v_t = v_t$

$\theta_{t+1} = \theta_t - v_t$

$0 < \eta^- < 1 < \eta^+$

> Onde $\theta$ é o parâmetro de network; $f$ a função de parâmetro de network; $t$ é o número de interações; $\alpha$ é o tamanho do passo/taxa de aprendizado; e o $mu$ o momentum.

## Rprop inicialização

* Inicia todas as atualizações com a iteração 0 até um valor constante.
> Se você indicar as duas taxas de aprendizado igual a 1, temos a regra de atualização de Manhattan.

$v_o = \delta$

* Rprop divide efetivamente o gradiente pela sua magnitude.
> Nunca atualize usando o gradiente, mas  isso pode ser um sinal.

## Problemas com Rprop

* Considerando um peso que é atualizado em 0.1 em nove *mini-batches* e -0.9 na décima *mini-batch*. SGD poderia manter esse peso aproximadamente onde ele começou, já Rprop iria incrementar o peso nove vezes por $\delta$ e na décima atualização atualizar diminuindo o peso em $\delta$;

> Atualização efetiva $9 \delta - \delta = 8 \delta$

* Dentro de *mini-batches* nos escalamos atualizações bem diferente.

## AdaGrad, RMSProp e Adam

In [0]:
def adagrad(args):
  grad_squared = 0
  while True:
    dx = compute_gradient(x)
    grad_squared += dx * dx # adiciona elementos escalando o gradiente
    # baseado no histórico de soma dos quadrados em cada dimensão
    x -= learning_rate * dx / (np.sqrt(grad_squared) + 1e-7)

def rmsprop(args):
  grad_squared = 0
  while True:
    dx = compute_gradient(x)
    grad_squared = decay_rate * grad_squared + (1 - decay_rate) * dx * dx
    x -= learning_rate * dx / (np.sqrt(grad_squared) + 1e-7)

def adam(args):
  first_moment = 0
  second_moment = 0
  while True:
    dx = compute_gradient(x)
    first_moment = beta1 * first_moment + (1 - beta1) * dx
    second_moment = beta2 * second_moment + (1 - beta2) * dx * dx # Momentum
    first_unbias = first_moment / (1 - beta1 ** t)
    second_unbias = second_moment / (1 - beta2 ** t) # correção de bias
    x -= learning_rate * first_moment / (np.sqrt(second_moment) + 1e-7) # AdaGrad / RMSProp

* Correção de bias para o fato do primeiro e segundo momento começarem com 0;

* Adam com `beta1 = .9`, `beta2 = .999` e `learning_rate = 1e-3` (ou 5e-4) é um bom ponto de start para vários modelos.

[ConvNetJS Trainer demo on MNIST](https://cs.stanford.edu/people/karpathy/convnetjs/demo/trainers.html)

* SGD, SGD+Momentum, Adagrad, RMSProp, Adam todos possuem **taxa de aprendizado** como hiper-parâmetro, qual é o melhor?

![loss_epoch](loss_epoch.png)

* A taxa de aprendizado cai com o tempo.

> Decaimento exponencial $\alpha = \alpha_0 e^{-kt}$

> Decaimento 1/t $\alpha = \frac{\alpha_o}{1 + kt}$

![learning](learning)

## Otimização de primeira ordem

1. Usa gradiente para aproximação linear;

2. Passo para minimizar a aproximação

![first_order](first_order.png)

## Otimização de segunda ordem

1. Usa o gradiente e Hessian para formar uma aproximação quadrática;

2. Passo para a mínima da aproximação

![second_order](second_order)

* Expansão de segunda ordem de Taylor:

$J(\theta) \approx J(\theta_0) + (\theta - \theta_0)^\top \nabla_{\theta}J(\theta_0) + \frac{1}{2}(\theta - \theta_0)^\top H(\theta - \theta_0)$

* Resolvendo para o ponto crítico obtemos a atualização para o parâmetro de Newton:

$\theta^* = \theta_0 - H^{-1} \nabla_\theta J(\theta_0$)

* O interessante sobre essa atualização é que não há hiper-parâmetros e nem taxa de aprendizado.

* Isso é ruim porque Hessian tem $\theta(N^2)$ elementos, ivertendo tem $\theta(N^3)$ onde $N$ é cerca de 10 ou 100 milhões;

* Métodos *Quasi-Newton* (BGFS o mais popular), em vez de inverter o Hessian ($\theta(N^3)$), aproxima o Hessian invertido com atualizações rank 1 frequentemente (\theta(N^2)$ cada);

* L-BFGS (versão de memória limitada do BFGS): Não salva a inversão inteira do Hessian.

> Normalmente funciona bem em uma *batch* inteira no modo determinístico. Por exemplo, se você tem uma função $f(x)$ determinístic então L-BFGS provavelmente trabalhará bem;

> Não transfere muito bem para o formato *mini-batch*, tem resultados ruins. Adaptando L-BFGS para altas escalas, o modo estocástico é uma área ativa de pesquisa.

## Na prática:

* **Adam** é uma ótima opção de escolha padrão na maioria dos casos;

* Se você pode usar uma *batch* inteira para atualizar então tente o **L-BFGS** (não esqueça de desativar todas as fontes de ruído).

## Babysitting learning

## Pesquisa de hiper-parâmetro

## Monitorar e visualizar a curva de perda

## Monitorar e visualizar a acurácia

## Rastrear a proporrção de atualização de pesos / magnitude do peso

## Além dos erros de treinamento

* Algoritmos bons de otimização ajudam a reduzir o erro de treinamento. Como reduzir o erro em dados novos?

* Como melhorar o desempenho de modelos singulares? **Regularização**

---

## Otimização de parâmetros em redes neurais

* Em *machine learning*, se começa definindo uma atividade e um modelo. O modelo consiste em uma arquitetura e em parâmetros. Para uma determinada arquitetura, os valores dos parâmetros determinam quão acurado o modelo executa a tarefa;

* Mas como se encontram bons valores? Definindo uma função de perda que avalia quão bem o modelo executa. O objetivo é minimizar a perda e assim encontrar valores de parâmetros que correspondem as predições com realismo.

### 1. Criando um problema de otimização

* A função de perda será diferente em diferentes atividades dependendo do que se quer como *output*. Como se define isso tem grande influência no quanto o modelo irá treinar e executar. Vejamos dois exemplos:

#### 1.1 Predição do preço de casas

* Digamos que sua atividade é predizer o preço de casas $y \in \mathbb{R}$ baseado em informações como área do piso, número de quartos, altura do teto. O quadrado da função de perda pode ser resumido por:

> Dados informações sobre a casa, o quadrado da diferença entre a predição e o preço real the que ser o menor possível.

$\mathcal{L} = ||y - \hat{y}||^2_2$

> Onde $\hat{y}$ é o preço predito e $y$ o preço real, conhecido como **verdade fundamental** (*ground truth*).

#### 1.2 Localização de objeto

* Num exemplo mais complexo precisamos encontrar um carro numa imagem que contém um. A função de perda é:

> Dada uma imagem contendo um carro, predizer qual caixa delimitadora (**bbox**) contém o carro. A caixa predita deve satisfazer o tamanho e a posição o carro verdadeiro o mais próximo possível.

$\mathcal{L} = (x - \hat{x})^2 + (y - \hat{y})^2 + (w - \hat{w})^2 + (h - \hat{h})^2$

> Centro do BBox: $(x - \hat{x})^2 + (y - \hat{y})^2$

> Largura/Altura do BBox: $(w - \hat{w})^2 + (h - \hat{h})^2$

* A função de perda depende de:
> 1. A predição do modelo depende dos valores do parâmetros (pesos) assim como as entradas (nessa caso, a imagem);
> 2. A **verdade fundamental** correspondente a entrada (rótulos, nesse caso caixas delimitadas).

### Função de custo

* Veja que a perda $\mathcal{L}$ recebe como entrada um exemplo único, então minimizar isso não garante parâmetros melhore para o modelo para os outros exemplos;

* É comum minimizar a média das perdas computadas em todo o dado de treinamento;

$\mathcal{J} = \frac{1}{m} \sum^m_{i=1} \mathcal{L}^{(i)}$

> Nós chamamos essa função de **custo**, onde $m$ é o tamanho dos dados de treinamento e $\mathcal{L}^{(i)}$ é a perda de um único dado de treinamento exemplo $x^{(i)}$ rotulado de $y^{(i)}.

### Visualizando a função de custo

* Para uma dada quantidade de exemplos e seus respectivos rótulos, a função de custo tem uma paisagem (gráfico) que varia como uma função do parâmetro da rede;

* È difícil visualizar essa paisagem, se existem mais de dois parâmetros. No entanto, essa paisagem existe e nosso objetivo é encontrar o ponto onde o valor da função de custo é (aproximadamente) mínimo;

* Atualizar os valores dos parâmetros vai mover o valor para mais próximo ou mais distante do nosso ponto mínimo alvo.

### O modelo contra a função de custo

* É importante distinguir a função $f$ que vai executar a ativadade (o modelo) e dá de saída um rótulo (como um **bbox** para um carro). Isso é definido pela arquitetura e os parâmetros, aproximando uma **função real** que executa a atividade. Valors otimizados de parâmetros vão permitir o modelo de executar a atividade com relativa acurácia;

* A função de custo tem como entrada os parâmetros e como saída o custo, avaliando quão bom os parâmetros são para realizar a atividade (nos dados de treinamento).

### Otimizando a função de custo

* Inicialmente não se sabe valores bons de parâmetro. No entando, temos a formula da funçao de custo. Minimizar a função de custo em teoria lhe ajudará a encontrar bons valores de parâmetros. O jeito de fazer isso é dar dados de treinamento ao modelo e ajustar os parâmetros interativamente para diminuir a função de custo o máximo possível;

* Em resumo, o jeito que se define a função de custo vai ditar o desempenho do modelo para a atividade. O diagrama abaixo ilustra o processo para encontrar o modelo que tem bom desempenho.

![car_diagram](car_diagram.png)


## Rodando o processo de otimização

* Para encontrar valores de parâmetro que obtenham o mínimo da função, podemos derivar uma solução de **forma aproximada** (*closed form*) por algebra ou aproximar isso usando um método interativo;

* Em *machine learning*, métodos interativos como **gradiente descedente** são frequentemente a única opção pela função de custo ser dependente de um grande número de variáveis e não nunca ter uma forma prática de encontrar uma forma aproximada de solução para o mínimo;

* Para o gradiente descedente, devemos inicializar os valores de parâmetro para ter um ponto de otimização. Então, se ajusta os valores de parâmetro interativamente para reduzir o valor da função de custo. Para cada interação, os valores de parâmetro são ajustados de acordo com a direção oposta do gradiente de custo (reduzindo custo);

$for\ x\ in\ dataset: $ - predição

$\hat{y} = model_W(x)$ - atualização dos parâmetros

$W = W - \alpha \frac{ \partial  \mathcal{J}(y, \hat{y})}{\partial{W}}$

> Onde $\hat{y}$ é a predição do modelo, dado a entrada $x$; $W$ é o parâmetro; $\frac{\partial \mathcal{J}}{\partial W}$ é o gradiente indicando a direção do $W$ para diminuir $\mathcal{J}$; $\alpha$ é a taxa de aprendizado onde é possível mexer para indicar o quanto você quer ajustar o valor de $W$ por interação.

* Veja que $\mathcal{J}$ recebe todo o dado de entrada e computar tudo isso poder ser lento. É normal minimizar a média da perda computada a partir de um conjunto de exemplos; para cada instância, $\mathcal{J}_{mini - batch} = \frac{1}{m_b} \sum^{m_b}_{i=1} \mathcal{L}^{(i)}$, reduzir essa função gera uma rápida atualização na direção de minimizar o **erro de treinamento**; $m_b$ é o **tamanho da amostra** (o parâmetro chave para tunar).

### Ajustando os hiper-parametros do gradiente descedente

* Para usar gradiente descendente devemos escolher valores para hiperparâmetros como taxa de aprendizado e tamanho da *batch*. Esses valores vão influenciar a otimização, então é importante escolhe-los apropriadamente.

> No site existe uma visualização para descobrir os parâmetros usados para criar os dados, usando o ponto de inicialização, taxa de aprendizado e tamanho da *batch*.

* Alguns aprendizados da visualização:
> 1. Mesmo escolhendo os melhores hiper-parametros, o modelo de treinamento nao vai corresponder exatamente a **verdade fundamental** (linhas azuis) porque os dados são uma representação da distribuição da **verdade fundamental**;
> 2. Quanto maior os dados de treinamento, mais próximo os parâmetros do seu modelo treinado estarão dos parâmetros usados para gerar os dados;
> 3. Se a taxa de aprendizdo é muito larga, seu algoritmo não irá covergir. Mas se é baixa ele irá covergir lentamente;
> 4. Se o ponto inicial (ponto vermelho) está perto da **verdade fundamental** e os hiper-parametros (taxa de aprendizado e tamanho da *batch*) são *tunados* apropriadamente, seu algoritmo irá covergir rapidamente.

### Inicialização

* Uma boa inicialização pode acelerar a otimização e permitir a convergência para um mínimo ou o melhor de vários mínimos.

* [Initializing Neural Networks](http://www.deeplearning.ai/ai-notes/initialization/)

### Taxa de aprendizado

* A taxa de aprendizado influencia a convergência da otimização, isso contrabalança a influência da função de custo na curvatura;

* De acordo com o gradiente descedente, a direção e magnitude da atualização do parâmetro são dados pela taa de aprendizado multiplicada pela inclinação da função custo num certo ponto $W$, especialmente: $\alpha \frac{\partial \mathcal{J}}{\partial W}$

> Se a taxa de aprendizado é pequena, atualizações são pequans e otimizações são lentas, especialmente se a curvatura de custo é baixa. Também é possível se acomodar num mínimo local ou plator;

> Se a taxa de aprendizado é muito alta, atualizações são altas e otimizações tendem a divergir, especialmente se a curvatura da função de custo
é alta;

> Se a taxa de aprendizado é escolhida bem, atualizações são apropriadas e a otimização tende a convergir para um bom grupo de parâmetros.

* No site tem-se uma visualização para encontrar o parâmetro correspondente ao custo mínimo usando gradiente descendente. O que ela ilustra:
> 1. O que faz uma boa taxa de aprendizado depende da curvatura da função custo;
> 2. O gradiente descendente cria uma aproximação linear da função custo num dado ponto. Então se move descendo e se aproximando da função custo;
> 3. Se o custo tem alta curvatura, quanto mais alto a taxa de aprendizado (passo) mais fácil o algoritmo ultrapassa;
> 4. Pequenos passos reduzem o problema anterior, mas diminui o aprendizado.

* É coum começar com uma alta taxa de aprendizado (entre 0.1 e 1) e ir diminuindo durante o treinamento. Escolhendo a diminuição (quão frequente? por quanto?) não é simples. Uma diminuição agressiva diminui o progresso em ireção ao ótimo, enquanto uma diminuição regular causa uma atualização caótica mas com pequenas melhoras.

* De fato, achando a melhor taxa de diminuição não é simples. N entanto, algoritmos de adaptação de taxa de aprendizado como **Momentum Adam** e **RMSprop** ajudam a ajustar a taxa de aprendizado durante o processo de  otimização.

## *Batch size*

* *Batch size* é o número de pontos nos dados usados para treinar um modelo em cada interação. *Batchs* tipicamente pequenas são: 32, 64, 128, 256, 512; enquanto grandes *batchs* podem ser centenas de eexemplos.

* Escolher o tamanh certo da *batch* é importante para garantir convergência da função custo e dos valores dos parâmetros, além da generalização do modelo. Algumas pesquisas indicam como fazer a escolha, mas não há consensus. Na prática se usa a **pesquisa por parâmetros**.

> O tamanho da *batch* determina a frequência de atualizações. Quanto menor a *batch* maior a quantidade de velocidade das atualizações;

> Quanto maior o tamanho da *batch*, mais acurado o gradiente do custo será em respeito aos parâmetros. Ou seja, a direção da atualização na maioria irá diminuir a inclinação local do custo;

> Tendo grandes *batchs*, mas não tão grande que não dê numa GPU, tende a melhorar a eficiência de paralização e pode acelerar o treino;

> Alguns autores (Keskar et al., 2016) sugerem que grandes *batchs* podem prejudicar a habilidade do modelo de generalizar, fazendo com que o modelo ache ótimos locais ou platores pobres.

* Escolhendo um tamanho de *batch*, existe um balancemento que depende do hardware disponível e a atividade que deve ser feita.

## Atualização interativa

* Agora que se tem ponto de início, taxa de aprendizado e o tamanho da *batch* é hora de atualizar os parâmetros interativamente para se mover em direção ao mínimo da função custo.

* O algoritmo de otimização é uma escolha central, pode-se testar vários otimizadores na visualização do site para ver os pros e contras de cada um.

* A ideia na visualização é brincar com os hiperparâmetros para encontras os valores dos parâmetros que minimizam a função custo. É possível escolher a função de custo e o ponto de início da otimização. Não existe modelo explicito, pode-se considerar que achando o mínimo da função custo é equivalente a encontrar o melhor modelo para a atividade. Para ser simples, o modelo só tem dois parâmetros e o tamnho da *batch* sempre é um.

## Escolha do otimizador

* A escolha do otimizador influencia tanto a velocidade da convergencia e se ocorre. Vária alternativas do gradiente descedente clássico foram desenvolvidas:

> **Gradiente descedente estocástico**: Pode usar paralelização com eficiência, mas é lento quando os dados são tão grandes que a GPU não consegue lidar. Usualmente converge mais rápido que o gradiente descedente clássico em grandes dados, porque atualizações são mais frequentes. É usualmente mais preciso sem usar todos os dados por ser frequentemente redundante. De todas as opções aqui é que usa menos memória.

> **Momentum**: Usualmente aumentaa velocidade de aprendizado com poucas mudanças na implementação. Usa mais memória que o gradiente descedente e menos que o RMSprop e Adam;

> **RMSprop**: O aprendizado adaptativo dele usualmente previne a diminuição ou aumento brusco da taxa de aprendizado. Ele mantém a taxa de aprendizado de *per-parametros*. Usa mais memória que os citados anteriormente e menos que o Adam;

> **Adam**: Os seus hiperparametros são constantes (predefinidos no artigo do algoritmo) não sendo necesário tunar. Ele executa uma taxa de aprendizado por anelamento com passos adaptativos. É o que mais usa memória dos citados, sendo o otimizador padrão em aprendizado de máquina.

* Métodos de otimização como **Adam** ou **RMSprop** executam bem na porção inicial do treinamento, mas eles generalizão pocuo nos últimos estágios quando comparados ao **gradiente descedente estocástico**.

## Conclusão

* Explorar métodos de otimização e valores de hiper-parâmetros podem ajudar a construir intuição para redes de otimização para suas atividades. Durante a pesquisa por hiper-parametros é importante entender intuitivimente a sensibilidade da otimização a taxa de aprendizado, tamanho da *batch*, otimizador, etc. Esse entendimento intuitivo combinado com o método correto (pesquisa aleatória ou otimização bayesiana) vão lhe ajudar a encontrar o modelo certo.

## Referências

* [Lecture 6 | Training Neural Networks I - 27:25](https://www.youtube.com/watch?v=wEoyxE0GP2M&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv&index=6)

* [System optimization in neural networks](http://www.deeplearning.ai/ai-notes/optimization/)